In [17]:
import praw
import pandas as pd
import datetime as dt
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

keys = pd.read_csv("./../Keys/Reddit Keys.csv")
reddit = praw.Reddit(client_id=keys.client_id[0],
                     client_secret=keys.client_secret[0],
                     user_agent=keys.user_agent[0])


In [18]:
print(reddit.read_only)  # Output: True

True


In [92]:
subreddit = reddit.subreddit('Conservative')

In [106]:
for submission in subreddit.hot(limit=10):
    print(submission.title)


President Benjamin Harrison explains why the Left hates the traditional family
Al Sharpton taking more than a million dollars from his “charity’ as salary.
Only a Muslim Democrat can get away with this s***: Ilhan Omar funneled another $150K to alleged lover’s consulting group
Obama Granted Clemency to Terrorists and Traitors, But We’re Supposed to be Angry at Trump's Pardons
Department of Justice Threatens to Arrest Oregon Judges Who Try to Keep Immigration Agents From Enforcing the Law
Ilhan Omar funneled another $150K to alleged lover’s consulting group
Poll: 53% of Black Voters, 60% of Other Minority Voters Say Media Are Trying to Help Impeach Trump
Democratic Representative Says He Will Vote NO on Impeachment, Reveals Other Dem Lawmakers “Quietly, Privately” Concerned About Schiff Show Trials
Portland - Deranged Anti-ICE Protester Sentenced To 3 Years In Prison After Attacking Disabled Man. Last Week another antifa terrorist received a six year prison sentence after wacking a guy 

In [94]:
top_subreddit = subreddit.top(limit=5000)


In [95]:
topics_dict = { "title":[], 
                "score":[], 
                "id":[], "url":[],  
                "comms_num": [], 
                "created": [], 
                "body":[]}


In [96]:
for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)



In [97]:
topics_data = pd.DataFrame({ "title":[], 
                "score":[], 
                "id":[], "url":[],  
                "comms_num": [], 
                "created": [], 
                "body":[]})


In [3]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)


In [99]:
_timestamp = topics_data["created"].apply(get_date)


In [100]:
_timestamp = topics_data["created"].apply(get_date)

topics_data = topics_data.assign(timestamp = _timestamp)


In [101]:
topics_data.head()

,title,score,id,url,comms_num,created,body,timestamp
0,So let me get this straight...,19572,5iugd6,http://i.imgur.com/L0p4pQA.jpg,3200,1.482010e+09,,2016-12-17 16:27:38
1,Why we won,15250,5c3xah,https://pbs.twimg.com/media/Cw1oX9aUQAA-rBU.jpg,1645,1.478756e+09,,2016-11-10 00:41:04
2,Ben Shapiro on the 4th of July...,14017,6la6tn,https://i.redd.it/ri98l0fdjn7z.jpg,1257,1.499236e+09,,2017-07-05 02:28:00
3,"Well, she's a guy, so...",12066,60k0bb,https://i.redd.it/kvriz780bnmy.jpg,1431,1.490080e+09,,2017-03-21 03:13:52
4,Reddit Admin u/spez Admits of Editing Users Co...,11301,5el3n6,https://i.redd.it/jt4fsjpargzx.png,610,1.479980e+09,,2016-11-24 04:29:49


In [102]:
len(topics_data)

998

In [103]:
topics_data.describe()

,score,comms_num,created
count,998.000000,998.000000,9.980000e+02
mean,2928.532064,299.720441,1.547567e+09
std,1384.685923,253.788856,2.531359e+07
min,1859.000000,26.000000,1.434479e+09
25%,2167.000000,153.250000,1.541131e+09
50%,2545.000000,234.000000,1.554972e+09
75%,3185.250000,352.000000,1.564978e+09
max,19572.000000,3200.000000,1.574026e+09


In [104]:
max(topics_data.timestamp)

Timestamp('2019-11-17 16:22:21')

In [105]:
min(topics_data.timestamp)

Timestamp('2015-06-16 14:31:13')

In [6]:

def make_request(uri, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1
    return fire_away(uri)


In [4]:
def pull_posts_for(subreddit, start_at, end_at):
    
    def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'created_utc': post['created_utc'],
            'prefix': 't4_'
        }, posts))
    
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
    
    post_collections = map_posts( \
        make_request( \
            URI_TEMPLATE.format( \
                subreddit, start_at, end_at, SIZE))['data'])
    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - (10)
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format( \
                    subreddit, new_start_at, end_at, SIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)
    return post_collections

In [5]:
def give_me_intervals(start_at, end_at, number_of_days_per_interval = 3):
            
    ## 1 day = 86400,
    period = (86400 * number_of_days_per_interval)
    end = start_at + period
    yield (int(start_at), int(end))
    padding = 1
    while end <= end_at:
        start_at = end + padding
        end = (start_at - padding) + period
        yield int(start_at), int(end)
## test out the solution,
start_at = math.floor(\
     (datetime.utcnow() - timedelta(days=365)).timestamp())
list(give_me_intervals(start_at, 7))


[(1544416684, 1544675884)]

In [89]:
subreddit = 'Conservative'
from datetime import date
end_at=date.fromisoformat('2018-11-06')
start_at=(end_at) - timedelta(days=30)
end_at_unix = time.mktime(end_at.timetuple())
start_at_unix = time.mktime(start_at.timetuple())

posts = []
for interval in give_me_intervals(start_at_unix, end_at_unix, 7):
    pulled_posts = pull_posts_for(
        subreddit, interval[0], interval[1])
    
    posts.extend(pulled_posts)
    time.sleep(.500)
## ~ 4306
print(len(posts))
## ~ 4306
print(len(np.unique([ post['id'] for post in posts ])))

9219
9202


In [8]:
election_dates = ['2018-11-06','2016-11-08',
                 '2014-11-04','2012-11-06']

In [110]:
subreddits = ['Liberal','Conservative','moderatepolitics']

In [142]:
posts = []
z=0
for x in election_dates:

    for y in subreddits:
        print(x, " : ", y)
        subreddit = y
        end_at=date.fromisoformat(x)
        start_at=(end_at) - timedelta(days=30)
        end_at_unix = time.mktime(end_at.timetuple())
        start_at_unix = time.mktime(start_at.timetuple())

        temp_posts = []
        for interval in give_me_intervals(start_at_unix, end_at_unix, 7):
            pulled_posts = pull_posts_for(
                subreddit, interval[0], interval[1])

            temp_posts.extend(pulled_posts)
            time.sleep(.500)
        
        if z==0:
            df = pd.DataFrame({'year':x,
                              'subreddit':y,
                              'posts':temp_posts})
            
            z=1
        else:
            df2 = pd.DataFrame({'year':x,
                              'subreddit':y,
                              'posts':temp_posts})
            
            df=df.append(df2,ignore_index=True)

            
   


2018-11-06  :  Liberal
2018-11-06  :  Conservative
2018-11-06  :  moderatepolitics
2016-11-08  :  Liberal
2016-11-08  :  Conservative
2016-11-08  :  moderatepolitics
2014-11-04  :  Liberal
2014-11-04  :  Conservative
2014-11-04  :  moderatepolitics
2012-11-06  :  Liberal
2012-11-06  :  Conservative
2012-11-06  :  moderatepolitics


In [143]:
df.head()

,year,subreddit,posts
0,2018-11-06,Liberal,"{'id': '9m2dtt', 'created_utc': 1538884997, 'p..."
1,2018-11-06,Liberal,"{'id': '9m2f5d', 'created_utc': 1538885384, 'p..."
2,2018-11-06,Liberal,"{'id': '9m2ixb', 'created_utc': 1538886524, 'p..."
3,2018-11-06,Liberal,"{'id': '9m2kyh', 'created_utc': 1538887168, 'p..."
4,2018-11-06,Liberal,"{'id': '9m3ygv', 'created_utc': 1538905752, 'p..."


In [144]:
print(len(df))

27830


In [145]:
df['subreddit'].value_counts()

Conservative        24175
Liberal              2916
moderatepolitics      739
Name: subreddit, dtype: int64

In [191]:
df['year'].value_counts()

2018-11-06    10308
2012-11-06     7319
2016-11-08     5603
2014-11-04     4600
Name: year, dtype: int64

In [ ]:
final_dict = { "subreddit":[],
            "year":[],
            "title":[], 
            "score":[], 
            "id":[], "url":[],  
            "comms_num": [], 
            "created": [], 
            "body":[]}

posts_from_reddit = []




In [210]:
## WARNING: REDDIT WILL THROTTLE YOU IF YOU ARE ANNOYING! BE KIND!

for x in range(int(len(final_dict['year'])),int(len(df))):
    submission = reddit.submission(id=df['posts'][x]['id'])
    posts_from_reddit.append(submission)
#    submission.comments.replace_more(limit=None)
#    for comment in submission.comments.list():
#        comments_from_reddit.append(comment)

    final_dict["title"].append(submission.title)
    final_dict["score"].append(submission.score)
    final_dict["id"].append(submission.id)
    final_dict["url"].append(submission.url)
    final_dict["comms_num"].append(submission.num_comments)
    final_dict["created"].append(submission.created)
    final_dict["body"].append(submission.selftext)
    final_dict['subreddit'].append(df['subreddit'][x])
    final_dict['year'].append(df['year'][x])

    time.sleep(0.5)
    
#    if x%400==0:
#        print(x, " posts completed")


In [204]:
sub_df = pd.DataFrame(final_dict)    

#make temp df
sub_df_v2 = sub_df

#get posted date, add to df
_timestamp = sub_df_v2["created"].apply(get_date)
sub_df_v2 = sub_df_v2.assign(timestamp = _timestamp)

sub_df_v2['election year']=sub_df_v2['year'].astype(str).str[0:4]


In [205]:
sub_df_v2

,subreddit,year,title,score,id,url,comms_num,created,body,timestamp,election year
0,Liberal,2018-11-06,Changing party affiliation in response to Kave...,13,9m2dtt,https://www.reddit.com/r/Liberal/comments/9m2d...,49,1.538914e+09,"Hey everyone,\n\nSo, I've long tried to be the...",2018-10-07 08:03:17,2018
1,Liberal,2018-11-06,My mother and I have decided to cut off commun...,1,9m2f5d,https://www.reddit.com/r/Liberal/comments/9m2f...,0,1.538914e+09,[removed],2018-10-07 08:09:44,2018
2,Liberal,2018-11-06,What can I do to help?,0,9m2ixb,https://www.reddit.com/r/Liberal/comments/9m2i...,10,1.538915e+09,[deleted],2018-10-07 08:28:44,2018
3,Liberal,2018-11-06,The current World Population is estimated to b...,4,9m2kyh,https://www.reddit.com/r/Liberal/comments/9m2k...,8,1.538916e+09,[deleted],2018-10-07 08:39:28,2018
4,Liberal,2018-11-06,"С кем поведешься, от того и наберешься",1,9m3ygv,https://www.reddit.com/r/Liberal/comments/9m3y...,0,1.538935e+09,[removed],2018-10-07 13:49:12,2018
...,...,...,...,...,...,...,...,...,...,...,...
27825,moderatepolitics,2012-11-06,Quotas Limiting Male Science Enrollment Could ...,1,12ydp9,http://www.openmarket.org/2012/07/10/quotas-li...,0,1.352559e+09,[deleted],2012-11-10 09:45:32,2012
27826,moderatepolitics,2012-11-06,"Veterans, facing an unforgiving and bureaucrat...",6,12yxvd,http://www.marketplace.org/topics/wealth-pover...,0,1.352598e+09,,2012-11-10 20:42:43,2012
27827,moderatepolitics,2012-11-06,13 things that would have passed the Senate if...,62,12z3p6,http://www.denverpost.com/politics-national/20...,11,1.352604e+09,,2012-11-10 22:25:30,2012
27828,moderatepolitics,2012-11-06,Why Americans Need Hate Like Air,10,12z5af,http://thoughtcatalog.com/2012/american-discor...,4,1.352606e+09,,2012-11-10 22:54:24,2012


In [206]:
len(sub_df)

27830

In [207]:
sub_df['year'].value_counts()

2018-11-06    10308
2012-11-06     7319
2016-11-08     5603
2014-11-04     4600
Name: year, dtype: int64

In [208]:
sub_df['subreddit'].value_counts()

Conservative        24175
Liberal              2916
moderatepolitics      739
Name: subreddit, dtype: int64

In [169]:
sub_df_2018 = sub_df.loc[sub_df['year']=='2018-11-06']

In [209]:
sub_df.to_csv("Reddit_Election_Posts.csv",index=False)

In [10]:
test_subreddits = ['Economics','arizonapolitics','wisconsinpolitics','politics']

In [9]:
election_dates = ['2018-11-06','2016-11-08',
                 '2014-11-04','2012-11-06']

In [19]:
posts = []
z=0
from datetime import date


for x in election_dates:

    for y in test_subreddits:
        print(x, " : ", y)
        subreddit = y
        end_at=date.fromisoformat(x)
        start_at=(end_at) - timedelta(days=30)
        end_at_unix = time.mktime(end_at.timetuple())
        start_at_unix = time.mktime(start_at.timetuple())

        temp_posts = []
        for interval in give_me_intervals(start_at_unix, end_at_unix, 7):
            pulled_posts = pull_posts_for(
                subreddit, interval[0], interval[1])

            temp_posts.extend(pulled_posts)
            time.sleep(.500)
        
        if z==0:
            df = pd.DataFrame({'year':x,
                              'subreddit':y,
                              'posts':temp_posts})
            
            z=1
        else:
            df2 = pd.DataFrame({'year':x,
                              'subreddit':y,
                              'posts':temp_posts})
            
            df=df.append(df2,ignore_index=True)

            

2018-11-06  :  Economics
2018-11-06  :  arizonapolitics
2018-11-06  :  wisconsinpolitics
2018-11-06  :  politics
2016-11-08  :  Economics
2016-11-08  :  arizonapolitics
2016-11-08  :  wisconsinpolitics
2016-11-08  :  politics
2014-11-04  :  Economics
2014-11-04  :  arizonapolitics
2014-11-04  :  wisconsinpolitics
2014-11-04  :  politics
2012-11-06  :  Economics
2012-11-06  :  arizonapolitics
2012-11-06  :  wisconsinpolitics
2012-11-06  :  politics


In [20]:
df.head()

,year,subreddit,posts
0,2018-11-06,Economics,"{'id': '9m2oso', 'created_utc': 1538888471, 'p..."
1,2018-11-06,Economics,"{'id': '9m4ize', 'created_utc': 1538913618, 'p..."
2,2018-11-06,Economics,"{'id': '9m4pty', 'created_utc': 1538915766, 'p..."
3,2018-11-06,Economics,"{'id': '9m54qw', 'created_utc': 1538919823, 'p..."
4,2018-11-06,Economics,"{'id': '9m5d3p', 'created_utc': 1538922000, 'p..."


In [21]:
print(len(df))

188499


In [22]:
df['subreddit'].value_counts()

politics             180608
Economics              7611
arizonapolitics         170
wisconsinpolitics       110
Name: subreddit, dtype: int64

In [23]:
df['year'].value_counts()

2012-11-06    65270
2016-11-08    65236
2018-11-06    44471
2014-11-04    13522
Name: year, dtype: int64

In [ ]:
final_test_dict = { "subreddit":[],
            "year":[],
            "title":[], 
            "score":[], 
            "id":[], "url":[],  
            "comms_num": [], 
            "created": [], 
            "body":[]}

posts_from_reddit = []

In [71]:

for x in range(int(len(final_test_dict['year'])),int(len(df))):
    submission = reddit.submission(id=df['posts'][x]['id'])
    posts_from_reddit.append(submission)

    final_test_dict["title"].append(submission.title)
    final_test_dict["score"].append(submission.score)
    final_test_dict["id"].append(submission.id)
    final_test_dict["url"].append(submission.url)
    final_test_dict["comms_num"].append(submission.num_comments)
    final_test_dict["created"].append(submission.created)
    final_test_dict["body"].append(submission.selftext)
    final_test_dict['subreddit'].append(df['subreddit'][x])
    final_test_dict['year'].append(df['year'][x])

    if x%1800==0:
        time.sleep(0.5)

    if x%10000==0:
        print(x, " posts completed")



140000  posts completed
150000  posts completed
160000  posts completed
170000  posts completed
180000  posts completed


In [72]:
len(final_test_dict['score'])

188499

In [73]:
sub_df_test = pd.DataFrame(final_test_dict)    

#make temp df
sub_df_test_v2 = sub_df_test

#get posted date, add to df
_timestamp = sub_df_test_v2["created"].apply(get_date)
sub_df_test_v2 = sub_df_test_v2.assign(timestamp = _timestamp)

sub_df_test_v2['election year']=sub_df_test_v2['year'].astype(str).str[0:4]



In [74]:
sub_df_test_v2['subreddit'].value_counts()

politics             180608
Economics              7611
arizonapolitics         170
wisconsinpolitics       110
Name: subreddit, dtype: int64

In [75]:
sub_df_test_v2['year'].value_counts()

2012-11-06    65270
2016-11-08    65236
2018-11-06    44471
2014-11-04    13522
Name: year, dtype: int64

In [76]:
sub_df_test_v2.tail()

,subreddit,year,title,score,id,url,comms_num,created,body,timestamp,election year
188494,politics,2012-11-06,Karl Rove and the election disaster,1,12zu0h,http://assertivetalk.com/?p=276,0,1.352635e+09,,2012-11-11 06:52:15,2012
188495,politics,2012-11-06,Yup,2,12zu51,http://i.imgur.com/lei5X.jpg,3,1.352635e+09,[deleted],2012-11-11 06:54:33,2012
188496,politics,2012-11-06,The Supreme Court Will Hear a Challenge to the...,40,12zu8d,http://www.theatlantic.com/national/archive/20...,0,1.352635e+09,,2012-11-11 06:56:10,2012
188497,politics,2012-11-06,Inside Team Romney’s whale of an IT meltdown |...,52,12zu94,http://arstechnica.com/information-technology/...,17,1.352635e+09,,2012-11-11 06:56:33,2012
188498,politics,2012-11-06,"Fox News: Dana Perino said she ""would stab a d...",1,12zubp,http://video.foxnews.com/v/1959436530001/elect...,0,1.352635e+09,,2012-11-11 06:58:05,2012


In [77]:
sub_df_test_v2.to_csv("reddit_test_posts.csv")